In [1]:
import cv2
import numpy as np
import math

cap = cv2.VideoCapture(0)
cap.set(3, 160)
cap.set(4, 120)

black_range = np.array([(0, 0, 170), (360, 50, 255)])
while True:
    ret, img = cap.read()
    
    img = cv2.bitwise_not(img) #img 색 반전
    mask = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #bgr에서 hsv로 변환
    mask = cv2.inRange(mask, black_range[0], black_range[1]) #black_range안에 있는것만 걸러낸다고 지정
    img = cv2.bitwise_and(img, img, mask=mask) #채로 걸러낸다.
    
    contours, hierarchy = cv2.findContours(mask, 1, cv2.CHAIN_APPROX_NONE)
    
    
    if len(contours) > 0 :
        c = max(contours, key=cv2.contourArea)
        M = cv2.moments(c)
        if M["m00"] !=0 :
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            rows,cols=img.shape[:2]
            [vx,vy,x,y]=cv2.fitLine(c,cv2.DIST_L2,0,0.01,0.01)
            y1=int((-x*vy/vx)+y)
            y2=int(((cols-x)*vy/vx)+y)
            rad = math.atan(float(y1-y2)/(cols-1))
            deg=round(rad*180/(np.pi),3)
            deg=str(deg)
            
            cv2.putText(img, deg, (100, 100), 0, 3, (0, 0, 255), 5)
            cv2.circle(img, (cx,cy), 5, (255,255,255), -1)
            cv2.line(img,(cx,cy-500),(cx,cy+500),(0,0,255),3)
            cv2.line(img,(0,y1),(cols-1,y2),(0,0,255),2)
            cv2.drawContours(img, c, -1, (0,255,0), 2)
    else :
        print("I don't see the line")
    cv2.imshow("Mask",mask)
    cv2.imshow("img",img)
    if cv2.waitKey(1) & 0xff == ord('q'):   # 1 is the time in ms
        break
cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 